# <center>The Battle of the Neighbourhood</center>

## Introduction
The battle of the neighbourhood is a capstone project for IBM Data Science Professional Cetrified for Data Science.

I have given my self the question of which region would be suited to set up an air conditioner companies in a region within New Zealand.  This will be based on comparision between the regions of Canterbury.

##  The business problem.

Which location in the two region have area lack converage for heat pump companies compare with PM10 data?

Which of the location have increasing incomes?  Leading to greater ability in purchasing heat pump as opposed to cheaper forms of heating requiring burning 
 
 ## Method

Software : 
Python 3.9.2 
JuptyerNotebook
Visual Studio Code


Data: 
I will be using PM10 from ECAN (Environment Canturbury).  Export data from Stats NZ and FourSquare API for Miscellous.



In [1]:
# Import the required libraries
import pandas as pd
import requests
import bs4 as soup
import io


Extract PM10 Data

PM10 is a fine particle that is created mainly from burning fuel sources.  In New Zealand most of the home heating is solid fuel heating woodburner(woodstove).  This is changing however due the stricter standards on air quality for city/town to prevent negative health effects.  

ECAN is the regulatory body for the province of Canturbury.  It task with monitoring and in forcing environmental standards.  As part of this it has a number of PM10 monitoring in it region 

https://www.ecan.govt.nz/







In [2]:
# ECAN Air Quality sites
ECAN_extract= []
ECANSiteList =   [7,2,9,4,5,1,10,64,77,3]
for Site in ECANSiteList:
    Extraction_Bay = requests.get('http://data.ecan.govt.nz/data/98/Air/Air%20quality%20data%20for%20a%20monitored%20site%20(Daily)/JSON?SiteId='+str(Site)+'&StartDate=01%2F01%2F2020&EndDate=01%2F01%2F2021')
    ECAN_extract.append(Extraction_Bay.json())



In [3]:
AirList = ['item',['DateTime']]

ECAN_list = []
for i in ECAN_extract:
    bay = i['data']['item'].copy()
    ECAN_list = ECAN_list+bay
    



ECANRaw = pd.json_normalize(ECAN_list)
ECAN = ECANRaw[['DateTime','StationName','PM10_x0020__x0028_ug_x002F_m3_x0029_']]
ECAN.columns = ['DateTime','StationName','PM10']
ECAN = ECAN[ECAN['PM10'].notnull()].copy()
ECAN['PM10'] = ECAN.PM10.astype(float).round(1)
ECAN.head()

,DateTime,StationName,PM10
0,2020-01-01,Ashburton,23.7
1,2020-01-02,Ashburton,18.8
2,2020-01-03,Ashburton,9.7
3,2020-01-04,Ashburton,11.5
4,2020-01-05,Ashburton,26.0


In [4]:
CLIENT_ID = 'BBRUYDTJVTG2IYAVIKCHLF1GPKO1WGVTZKNFZJGLQFJKLLRQ' # your Foursquare ID
CLIENT_SECRET = 'GWHGWMRKZT5TS1UNTRAMKESJXDMTCIBJUIDCSF4DDIR5O1IY' # your Foursquare Secret
ACCESS_TOKEN = "VPMLHSDIANMBWYCWV4CI04RSLCAPIGLJR1GRSPTKPPF2ECT0" # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
b


Your credentails:
CLIENT_ID: BBRUYDTJVTG2IYAVIKCHLF1GPKO1WGVTZKNFZJGLQFJKLLRQ
CLIENT_SECRET:GWHGWMRKZT5TS1UNTRAMKESJXDMTCIBJUIDCSF4DDIR5O1IY


In [ ]:
Loction = unique(ECAN.StationName)

In [ ]:
Dict = {'Location' = ['ChristChurch','Asburton',''] 'Lat' =[43.5320],'Long' = [172.6306]}
LatLongdf = pd.DataFrame(data= Dict)